In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

In [2]:
def flux_blr(amp_Hb1,amp_Hb2,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = amp_Hb1*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = amp_Hb2*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def flux_FeII_5018(amp_FeII_5018_1,amp_FeII_5018_2,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = amp_FeII_5018_1*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = amp_FeII_5018_2*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def Hb_blr_lum(z,flux):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return Hb_lum
    
def ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br):
    ratio = (vel_offset/vel_sigma_OIII_br)
    return ratio

def line_shape_par(z,wav_min,wav_max,cor_popt):
    c = 3*(10**5)
    wave = np.arange(wav_min,wav_max,0.02) 
    cumsum = np.cumsum(OIII_wo_cont(wave,*cor_popt))
   # print (cumsum)
    norm_sum=cumsum/cumsum[-1]
    
    select = (norm_sum>0.1) & (norm_sum<0.9)
    w80_spec = wave[select][-1]-wave[select][0]
 
    select =  (norm_sum>0.05) & (norm_sum<0.5)
    v5_spec = wave[select][0]-wave[select][-1]

    [w80,v5] = [((w80_spec)/5006.8)*(c/(1+z)),((v5_spec)/5006.8)*(c/(1+z))]
    return w80,v5


def blr_sigma(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2):
    c = 300000
    k = 1+z
    select = (mini_wave>4750*k) & (mini_wave < 5090*k)
    m_0 = np.sum(double_BLR(mini_wave[select],amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))
    m_1 = np.sum(mini_wave[select]*double_BLR(mini_wave[select],amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))/m_0
    m_2 = np.sqrt(np.sum(double_BLR(mini_wave[select],amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2)*(mini_wave[select]-m_1)**2)/m_0)
    sigma_Hb_blr = c*(m_2)/m_1
    return sigma_Hb_blr

def blr_FWHM(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2):
    c = 300000
    k = 1+z
    select = (mini_wave>4750*k) & (mini_wave<5090*k)
    (wav,dat) = (mini_wave[select],double_BLR(mini_wave[select],amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))
    #print (wav[select])
    amp = np.max(dat)
    select = (dat>amp/2) & (dat<amp)
    fwhm = wav[select][-1] - wav[select][0]
    fwhm = (fwhm/4861.33)*(c/(1+z))
    return fwhm

def par5100(z,m,c,wavelength=5100):
    k = 1 + z
    flux_den = (m*wavelength*k/1000.0) + c
    flux = wavelength*flux_den*(10**(-16))
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    log_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    lum = 10**log_lum
    return flux_den,flux,lum
    
def log_bolometric_luminosity(lum):
    L = 9.0*lum
    log_L = np.log10(L)
    return log_L

def logMBH(Hb_fwhm,lum_5100):
    logMBH=np.log10((Hb_fwhm/1000.)**2*(lum_5100/1.e44)**0.5)+6.91 
    return logMBH

def edd_lum(log_MBH):
    lum = 1.25*(10**38)*(10**log_MBH)
    return lum

def flux_lum_OIII_spectro(z,amp_OIII5007,amp_OIII5007_br,sigma_OIII_5007,sigma_OIII_5007_br):
    flux_5007_core = amp_OIII5007*(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007))
    flux_5007_wing = amp_OIII5007_br*(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007_br)) 
    flux = (flux_5007_core + flux_5007_wing)*(10**(-16))
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    log_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return flux,log_lum
     
def par_tab(obj,kin_par,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'w80':0,'v5':1,'vel_offset':2,'vel_sigma_OIII_br':3,'ratio':4,'blr_flux':5,'log_Hb_lum':6,'Hb_sigma':7,'Hb_fwhm':8,'log_L_5100':9,'log_MBH':10,'log_L_edd':11,'log_L_OIII':12,'log_Rout':13}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[kin_par[column_names[key]]]))
       # columns.append(fits.Column(name=key+'_err',format='E',array=[kin_par_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hduk = fits.BinTableHDU.from_columns(coldefs)
    hduk.writeto('%s/%s/%s.kinpar_fixkin_central.fits'%(destination_path_cube,obj,obj),overwrite=True)   
    
def source_par_tab(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s.kinpar_fixkin_central.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s.kinpar_fixkin_central_tab.fits'%(destination_path_cube,obj,obj),overwrite=True)
    
def nuclear_tab(obj,nuclear_par,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'fOIII_erg/s/cm2':0,'logL_OIII_erg/s':1,'f5100_erg/s/cm2/A':2,'loglamL5100_erg/s':3,'logLbol_erg/s':4,'logLedd_erg/s':5,'FWHM_Hb_km/s':6,'sig_Hb_km/s':7,'log_MBH':8,'frac_edd':9}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[nuclear_par[column_names[key]]]))
      #  columns.append(fits.Column(name=key+'_err',format='E',array=[nuclear_par_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdun = fits.BinTableHDU.from_columns(coldefs)
    hdun.writeto('%s/%s/%s_nucleartab.fits'%(destination_path_cube,obj,obj),overwrite=True)      

def source_nuclear_table(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_nucleartab.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_nucleartab.fits'%(destination_path_cube,obj,obj),overwrite=True) 

In [3]:
def algorithm_script(obj,z,broad2,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
    (amp_Hb1,amp_Hb2,amp_Fe5018_1,amp_Fe5018_2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c) = par_spectro(obj)
    (amp_Hb1_err,amp_Hb2_err,amp_Fe5018_1_err,amp_Fe5018_2_err,vel_Hb1_err,vel_Hb2_err,vel_sigma_Hb1_err,vel_sigma_Hb2_err,amp_OIII5007_err,vel_OIII_err,vel_sigma_OIII_err,amp_OIII5007_br_err,vel_OIII_br_err,vel_sigma_OIII_br_err,m_err,c_err) = par_spectro_err(obj)

    ############################## Get the parameters#################################################
    sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
    sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
    v_out = vel_OIII - vel_OIII_br
    flux = flux_blr(amp_Hb1,amp_Hb2,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
    flux_FeII = flux_FeII_5018(amp_Fe5018_1,amp_Fe5018_2,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
    Hb_lum = Hb_blr_lum(z,flux)
    Hb_sigma = blr_sigma(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2)
    Hb_fwhm = blr_FWHM(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2)
    (flux_den_5100,flux_5100,lum_5100) = par5100(z,m,c,wavelength=5100)
    log_lum_5100 = np.log10(lum_5100)
    log_lum_bol = log_bolometric_luminosity(lum_5100)
    lum_bol = 10**(log_lum_bol)
    log_MBH = logMBH(Hb_fwhm,lum_5100)
    L_edd = edd_lum(log_MBH)
    log_L_edd = np.log10(L_edd)
    frac_edd = (lum_bol/L_edd)              
    cor_popt = np.array([amp_OIII5007,amp_OIII5007_br,vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br])
    print (frac_edd)
    (wav_min,wav_max) = wavlim(vel_OIII,vel_OIII_br)
    (w80,v5) = line_shape_par(z,wav_min,wav_max,cor_popt)
    ratio = ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br)
    sigma_OIII_5007 = line_width_recons(vel_sigma_OIII,rest_line=5006.8,inst_res_fwhm=0)
    sigma_OIII_5007_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0) 
    (flux_OIII,log_lum_OIII) = flux_lum_OIII_spectro(z,amp_OIII5007,amp_OIII5007_br,sigma_OIII_5007,sigma_OIII_5007_br)
    #print (w80,log_lum_OIII,Hb_lum)
    log_Rout = 0.28*log_lum_OIII - 11.27
    kin_par = [w80,v5,v_out,vel_sigma_OIII_br,ratio,flux,Hb_lum,Hb_sigma,Hb_fwhm,lum_5100,log_MBH,L_edd,log_lum_OIII,log_Rout]
    nuclear_par = [flux_OIII,log_lum_OIII,flux_den_5100,log_lum_5100,log_lum_bol,log_L_edd,Hb_fwhm,Hb_sigma,log_MBH,frac_edd]
    ############################################################################################################
    
    spec_par = [amp_Hb1,amp_Hb2,amp_Fe5018_1,amp_Fe5018_2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c]
    spec_err = [amp_Hb1_err,amp_Hb2_err,amp_Fe5018_1_err,amp_Fe5018_2_err,vel_Hb1_err,vel_Hb2_err,vel_sigma_Hb1_err,vel_sigma_Hb2_err,amp_OIII5007_err,vel_OIII_err,vel_sigma_OIII_err,amp_OIII5007_br_err,vel_OIII_br_err,vel_sigma_OIII_br_err,m_err,c_err]
   
    par_tab(obj,kin_par)
    #print (nuclear_par)
    nuclear_tab(obj,nuclear_par)
    source_nuclear_table(obj)
    source_par_tab(obj)

In [4]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}
        
objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1810
0.007935054100903928
HE0021-1819
0.01943863206963292
HE0040-1105
0.04303721638448935
HE0108-4743
0.032167289929816684
HE0114-0015
0.027181470087222614
HE0119-0118
0.13047255098002605
HE0212-0059
0.002849435593410127
HE0224-2834
0.02414589933157242
HE0227-0913
0.28361297318596534
HE0232-0900
0.06416415554736796
HE0253-1641
0.048975845383151834
HE0345+0056
1.8690353432514935
HE0351+0240
0.8720707394921967
HE0412-0803
0.04056065518040632
HE0429-0247
0.4973646162626058
HE0433-1028
0.11455356559866432
HE0853+0102
0.012233483500510288
HE0934+0119
0.28966702566559926
HE1011-0403
0.055334947564472076
HE1017-0305
0.02848382380569633
HE1029-1831
0.03481861326441414


/home/mainak/Downloads/func.py:173: RuntimeWarning: divide by zero encountered in true_divide
  line = (amplitude)*exp(-(wave-(rest_wave*(1+redshift(vel))))**2/(2*(line_width_recons(vel_sigma, rest_wave))**2))


HE1107-0813
0.2049344914968828
HE1108-2813
0.04106381754005824
HE1126-0407
0.6277276866026776
HE1237-0504
0.007677770681406955
HE1248-1356
0.002214145172934753
HE1330-1013
0.015328756713215656
HE1353-1917
0.03167264872271543
HE1417-0909
0.03759990559532615
HE2128-0221
0.08718427155171747
HE2211-3903
0.015349055866568139
HE2222-0026
0.1576034960972461
HE2233+0124
0.03297185773557723
HE2302-0857
0.04937387167920935


In [5]:
a = [1,2,3]
b = [0.01,0.02,0.03]
np.random.normal(a,b)

array([1.00925688, 1.98221733, 2.91364715])

In [9]:
[np.random.normal(1,0.01),np.random.normal(2,0.01),np.random.normal(3,0.03)]

[1.0052235959117686, 2.0000030790547565, 3.01121461795282]

In [8]:
10**(-2.31)

0.004897788193684461

In [ ]:
1.9